In [ ]:
import os, json, time, math
import numpy as np
import scipy.io
from scipy.io import savemat
from sklearn.mixture import GaussianMixture
import gurobipy
from datetime import datetime
from scipy.stats import multivariate_normal as mvn
from scipy.special import logsumexp

# ====== CONFIG ===============================================================
Mode = "GMM"   # "Ind" | "DD" | "DRO" | "GMM"
ETA = 100     # DRO/GMM-DRO ambiguity size (set 0.0 for pure GMM)
NUM_ITER = 10  # Forward/Backward iterations
NORMALIZE = True  

GMM_COMPONENTS = 4
GMM_COVARIANCE_TYPE = "diag"  
COV_JITTER = 1e-8
LOG_EPS = 1e-12
# ============================================================================

np.random.seed(42)

eps = 1e-12
band_mult = 1.5 

w_floor = 1e-6
UNMET_UNIFORM_IF_DRO = False

gmm_joint_params = None   
per_t_cache = None    

def get_kernel_bandwidth_ours(samples, method):
    N, K = samples.shape
    S = np.cov(samples.T, ddof=1)
    if method == "exp":
        H = ((4 / ((2*K + 1) * N)) ** (2/(K+4))) * band_mult
    elif method == "epa":
        c = np.pi**(K/2) / math.gamma(K/2 + 1)
        H = (((8*K*(K+2)*(K+4)*(2*np.sqrt(np.pi))**K) / (N*(2*K+1)*c)) ** (2/(K+4)))
    return float(H), S

def get_kernel_likelihood_ours(x, y, H, S, method):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    diff = x - y
    Sinv = np.linalg.pinv(S, rcond=1e-12)
    d2 = float(diff.T @ Sinv @ diff)
    d2 = max(d2, 0.0)
    H = float(H)
    if method == "exp":
        val = -0.5 * math.sqrt(d2 / H) if H > 0 else -np.inf
        return math.exp(val)
    elif method == "epa":
        val = 1.0 - d2 / H if H > 0 else 0.0
        return max(val, 0.0)
    else:
        return 0.0

def create_sample_path(N, T, transition_matrix):
    sample_path = []
    for t in range(T):
        if t == 0:
            idx = np.random.choice(range(N), p=transition_matrix[t][0])
        else:
            idx = np.random.choice(range(N), p=transition_matrix[t][sample_path[-1]])
        sample_path.append(idx)
    return sample_path

def update_storage_and_revenue(x, price, power, curr_storage, cap, rho, rho_c, rho_d):
    revenue = float(np.dot(price, x))
    for h in range(24):
        curr_storage = rho * curr_storage  
        if power[h] >= x[h]:
            curr_excess = power[h] - x[h]
            if (curr_storage[2] + rho_c[2]*curr_excess >= cap[2]):
                curr_excess = curr_excess - 1/rho_c[2]*(cap[2] - curr_storage[2])
                curr_storage[2] = cap[2]
                if (curr_storage[1] + rho_c[1]*curr_excess >= cap[1]):
                    curr_excess = curr_excess - 1/rho_c[1]*(cap[1] - curr_storage[1])
                    curr_storage[1] = cap[1]
                    if (curr_storage[0] + rho_c[0]*curr_excess >= cap[0]):
                        curr_excess = curr_excess - 1/rho_c[0]*(cap[0] - curr_storage[0])
                        curr_storage[0] = cap[0]
                    else:
                        curr_storage[0] = curr_storage[0] + rho_c[0]*curr_excess
                else:
                    curr_storage[1] = curr_storage[1] + rho_c[1]*curr_excess
            else:
                curr_storage[2] = curr_storage[2] + rho_c[2]*curr_excess
        else:
            curr_short = x[h] - power[h]
            if (curr_short <= curr_storage[2]*rho_d[2]):
                curr_storage[2] = curr_storage[2] - 1/rho_d[2]*curr_short
            else:
                curr_short = curr_short - curr_storage[2]*rho_d[2]
                curr_storage[2] = 0
                if (curr_short <= curr_storage[1]*rho_d[1]):
                    curr_storage[1] = curr_storage[1] - 1/rho_d[1]*curr_short
                else:
                    curr_short = curr_short - curr_storage[1]*rho_d[1]
                    curr_storage[1] = 0
                    if (curr_short <= curr_storage[0]*rho_d[0]):
                        curr_storage[0] = curr_storage[0] - 1/rho_d[0]*curr_short
                    else:
                        curr_short = curr_short - curr_storage[0]*rho_d[0]
                        curr_storage[0] = 0
                        revenue = revenue - 2 * price[h] * curr_short
    return revenue, curr_storage

def save_quarter_result(base, q_idx, Mode, N, band_mult, eta, pvs_OOS):
    pvs = np.asarray(pvs_OOS, dtype=float)
    qbase = f"{base}_q{q_idx}"
    summary = {
        "Mode": Mode, "MODE": Mode, "N": int(N),
        "band_mult": float(band_mult), "eta": float(eta),
        "quarter": int(q_idx),
        "count": int(pvs.size),
        "mean": float(pvs.mean()) if pvs.size else None,
        "var":  float(pvs.var())  if pvs.size else None,
        "std":  float(pvs.std())  if pvs.size else None,
        "p10":  float(np.percentile(pvs, 10)) if pvs.size else None,
        "p90":  float(np.percentile(pvs, 90)) if pvs.size else None,
    }
    with open(qbase + "_summary.json", "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    with open(base + "_summary.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(summary, ensure_ascii=False) + "\n")

def _split_blocks(mu_k, Sig_k, dim_s):
    K, D = mu_k.shape
    dim_xi = D - dim_s
    mu_s   = mu_k[:, :dim_s]
    mu_xi  = mu_k[:, dim_s:]
    S_ss   = Sig_k[:, :dim_s, :dim_s].copy()
    S_sxi  = Sig_k[:, :dim_s, dim_s:].copy()
    S_xis  = Sig_k[:, dim_s:, :dim_s].copy()
    S_xixi = Sig_k[:, dim_s:, dim_s:].copy()
    return mu_s, mu_xi, S_ss, S_sxi, S_xis, S_xixi, dim_xi

def _posterior_k_given_xi(xi_mat, p_k, mu_xi, S_xixi, jitter=COV_JITTER):
    N = xi_mat.shape[0]
    K = p_k.shape[0]
    log_r = np.zeros((N, K))
    for k in range(K):
        S = S_xixi[k].copy()
        S[np.diag_indices_from(S)] += jitter
        log_r[:, k] = np.log(np.maximum(p_k[k], LOG_EPS)) + mvn.logpdf(xi_mat, mean=mu_xi[k], cov=S)
    log_r = log_r - logsumexp(log_r, axis=1, keepdims=True)
    return np.exp(log_r)  # (N,K)

def _precompute_conditional_blocks(mu_s, mu_xi, S_ss, S_sxi, S_xis, S_xixi, jitter=COV_JITTER):
    K = mu_s.shape[0]
    Kmat  = np.zeros_like(S_sxi)
    S_scond = np.zeros_like(S_ss)
    S_xixi_inv = np.zeros_like(S_xixi)
    for k in range(K):
        Sxx = S_xixi[k].copy()
        Sxx[np.diag_indices_from(Sxx)] += jitter
        inv = np.linalg.inv(Sxx)
        S_xixi_inv[k] = inv
        Kmat[k] = S_sxi[k] @ inv
        S_scond[k] = S_ss[k] - S_sxi[k] @ inv @ S_xis[k]
        S_scond[k][np.diag_indices_from(S_scond[k])] += jitter
    return Kmat, S_scond, S_xixi_inv

# ============================================================================
if __name__ == "__main__":
    N = 39
    T = 7            

    ts = time.strftime("%Y%m%d-%H%M%S")
    outdir = "Results"
    os.makedirs(outdir, exist_ok=True)
    base = f"{outdir}/OH_sddp_{Mode.lower()}_{ts}"

    eta = float(ETA if Mode in {"DRO", "GMM"} else 0.0)
    eta_save = eta

    for loc in ['data/wind_data/OH_all.mat']:
        mat_data  = scipy.io.loadmat(loc)
        mat_price = scipy.io.loadmat('data/wind_data/price_all.mat')

        for q in [1,2,3,4]:
            band_mult = 0.75 if q == 1 else 1.5

            weekly_wind  = mat_data['weekly_wind'][0][q-1]   
            weekly_power = mat_data['weekly_power'][0][q-1]
            weekly_price = mat_price['weekly_price'][0][q-1]

            nData = weekly_price.shape[0]
            nOOS = nData - N

            def split_daily(arr):
                daily = np.zeros((T+1, arr.shape[0], 24), dtype=float)
                for t in range(T+1):
                    sl = slice(t*24, (t+1)*24)
                    daily[t] = arr[:, sl]
                return daily

            daily_wind_all  = split_daily(weekly_wind)
            daily_power_all = split_daily(weekly_power)
            daily_price_all = split_daily(weekly_price)

            daily_wind      = daily_wind_all[:, :N, :]
            daily_power     = daily_power_all[:, :N, :]
            daily_price     = daily_price_all[:, :N, :]

            daily_wind_OOS  = daily_wind_all[:, N:, :]
            daily_power_OOS = daily_power_all[:, N:, :]
            daily_price_OOS = daily_price_all[:, N:, :]

            if NORMALIZE:
                scale = np.percentile(daily_price, 90)
                scale = float(scale if scale > 0 else 1.0)
                daily_price     /= scale
                daily_price_OOS /= scale

            if Mode in {"DRO", "GMM"}:
                eta = ETA / (scale**2) if NORMALIZE else float(ETA)
                eta_save = eta  

            wind_H_ours = np.zeros((T+1, 1), dtype=float)
            wind_S      = np.zeros((T+1, 24, 24), dtype=float)
            for t in range(T+1):
                H, S = get_kernel_bandwidth_ours(daily_wind[t], "exp")
                wind_H_ours[t] = H
                wind_S[t]      = S

            weightNW = np.zeros((T, N, N), dtype=float)

            global gmm_joint_params, per_t_cache
            gmm_joint_params = None
            per_t_cache = [None] * T

            # ------------------------------------------------
            def compute_weights_Ind():
                for t in range(T):
                    weightNW[t, :, :] = 1.0 / N

            def compute_weights_DD_or_DRO():
                for t in range(T):
                    for i in range(N):
                        for j in range(N):
                            weightNW[t, i, j] = get_kernel_likelihood_ours(
                                daily_wind[t, i, :], daily_wind[t, j, :],
                                wind_H_ours[t], wind_S[t], "exp"
                            )
                    s = weightNW[t].sum(axis=1, keepdims=True) + eps
                    weightNW[t] /= s

            def compute_weights_GMM_density_ratio():
                global gmm_joint_params, per_t_cache
                if per_t_cache is None:
                    per_t_cache = [None] * T

                Z_list = []
                xi_by_t = []
                s_by_t  = []
                for t in range(T):
                    xi_mat = np.asarray(daily_wind[t, :N, :], dtype=float)
                    s_mat  = np.asarray(daily_wind[t+1, :N, :], dtype=float) 
                    Z_list.append(np.concatenate([s_mat, xi_mat], axis=1))   
                    xi_by_t.append(xi_mat)
                    s_by_t.append(s_mat)
                Z_big = np.vstack(Z_list) 
                D = Z_big.shape[1]        

                gmm = GaussianMixture(
                    n_components=GMM_COMPONENTS,
                    covariance_type=GMM_COVARIANCE_TYPE, 
                    reg_covar=1e-6,
                    random_state=42
                ).fit(Z_big)
                p_k   = gmm.weights_.astype(float)       
                mu_k  = gmm.means_.astype(float)        
                Sig_k = gmm.covariances_                

                Kc = p_k.shape[0]
                if GMM_COVARIANCE_TYPE == "full":
                    Sig_full = Sig_k.astype(float)
                elif GMM_COVARIANCE_TYPE == "diag":
                    Sig_full = np.zeros((Kc, D, D), dtype=float)
                    for k in range(Kc):
                        Sig_full[k] = np.diag(Sig_k[k].astype(float))
                elif GMM_COVARIANCE_TYPE == "tied":
                    Sig_full = np.tile(Sig_k.astype(float)[None, :, :], (Kc, 1, 1))
                elif GMM_COVARIANCE_TYPE == "spherical":
                    Sig_full = np.zeros((Kc, D, D), dtype=float)
                    for k in range(Kc):
                        Sig_full[k] = np.eye(D) * float(Sig_k[k])
                else:
                    raise ValueError(f"Unsupported covariance_type: {GMM_COVARIANCE_TYPE}")

                mu_s, mu_xi, S_ss, S_sxi, S_xis, S_xixi, dim_xi = _split_blocks(mu_k, Sig_full, dim_s=24)
                Kmat, S_scond, S_xixi_inv = _precompute_conditional_blocks(mu_s, mu_xi, S_ss, S_sxi, S_xis, S_xixi)

                for t in range(T):
                    xi_mat = xi_by_t[t]
                    r_k_xi = _posterior_k_given_xi(xi_mat, p_k, mu_xi, S_xixi)
                    per_t_cache[t] = {"xi_mat": xi_mat, "s_mat": s_by_t[t], "r_k_xi": r_k_xi}

                for t in range(T):
                    xi_mat = per_t_cache[t]["xi_mat"] 
                    s_mat  = per_t_cache[t]["s_mat"]   
                    r_k_xi = per_t_cache[t]["r_k_xi"]  

                    log_f_s = np.zeros(N)
                    for j in range(N):
                        terms = []
                        for k in range(Kc):
                            Sss = S_ss[k] + COV_JITTER*np.eye(24)
                            terms.append(np.log(np.maximum(p_k[k], LOG_EPS)) +
                                         mvn.logpdf(s_mat[j], mean=mu_s[k], cov=Sss))
                        log_f_s[j] = logsumexp(np.array(terms))

                    W = np.zeros((N, N), dtype=float)
                    for i in range(N):
                        mu_cond = np.stack([
                            mu_s[k] + Kmat[k] @ (xi_mat[i] - mu_xi[k]) for k in range(Kc)
                        ], axis=0)  
                        for j in range(N):
                            log_terms = np.zeros(Kc)
                            for k in range(Kc):
                                log_terms[k] = (np.log(np.maximum(r_k_xi[i, k], LOG_EPS)) +
                                                mvn.logpdf(s_mat[j], mean=mu_cond[k], cov=S_scond[k]))
                            W[i, j] = np.exp(logsumexp(log_terms) - log_f_s[j])

                    row_sum = W.sum(axis=1, keepdims=True)
                    row_sum[row_sum <= 0.0] = 1.0
                    weightNW[t] = W / row_sum

                gmm_joint_params = {
                    "p_k": p_k, "mu_s": mu_s, "mu_xi": mu_xi,
                    "S_ss": S_ss, "S_scond": S_scond, "Kmat": Kmat,
                    "S_xixi": S_xixi
                }

            if Mode == "Ind":
                compute_weights_Ind()
            elif Mode == "DD":
                compute_weights_DD_or_DRO()
            elif Mode == "DRO":
                compute_weights_DD_or_DRO()
            elif Mode == "GMM":
                compute_weights_GMM_density_ratio()
            else:
                raise ValueError(f"Unknown Mode: {Mode}")

            dep_transition_matrix = [0 for _ in range(T+1)]
            for t in range(T+1):
                if t == 0:
                    dep_transition_matrix[t] = [[1/N for _ in range(N)]]
                else:
                    dep_transition_matrix[t] = [[weightNW[t-1, i, j] for j in range(N)] for i in range(N)]

            ind_transition_matrix = [0 for _ in range(T+1)]
            for t in range(T+1):
                if t == 0:
                    ind_transition_matrix[t] = [[1/N for _ in range(N)]]
                else:
                    ind_transition_matrix[t] = [[1/N for _ in range(N)] for _ in range(N)]

            Xi_price      = [[[0.0]*24 for _ in range(T+1)] for _ in range(N)]
            Xi_power      = [[[0.0]*24 for _ in range(T+1)] for _ in range(N)]
            Xi_price_OOS  = [[[0.0]*24 for _ in range(T+1)] for _ in range(nOOS)]
            Xi_power_OOS  = [[[0.0]*24 for _ in range(T+1)] for _ in range(nOOS)]

            for idx in range(N):
                for t in range(T+1):
                    Xi_price[idx][t] = list(daily_price[t, idx, :])
                    Xi_power[idx][t] = list(daily_power[t, idx, :])

            for idx in range(nOOS):
                for t in range(T+1):
                    Xi_price_OOS[idx][t] = list(daily_price_OOS[t, idx, :])
                    Xi_power_OOS[idx][t] = list(daily_power_OOS[t, idx, :])

            Markov_states_price = [ [tmp[t] for tmp in Xi_price] for t in range(T+1) ]
            Markov_states_power = [ [tmp[t] for tmp in Xi_power] for t in range(T+1) ]
            Markov_states_price_OOS = [ [tmp[t] for tmp in Xi_price_OOS] for t in range(T+1) ]
            Markov_states_power_OOS = [ [tmp[t] for tmp in Xi_power_OOS] for t in range(T+1) ]

            transition_matrix = ind_transition_matrix if Mode == "Ind" else dep_transition_matrix

            OutputFlag = 0
            N_ins = len(Markov_states_power[1])     
            T_ = len(Markov_states_power) - 1

            M = [[None for _ in range(N_ins)] for _ in range(T_)]

            cap   = np.array([5000, 2000, 1000], dtype=float)
            rho   = np.array([0.98, 0.99, 0.995], dtype=float)
            rho_c = np.array([0.8, 0.9, 1.0], dtype=float)
            rho_d = np.array([0.8, 0.9, 1.0], dtype=float)
            init_level = np.array([5000, 2000, 1000], dtype=float)

            start_init = datetime.now()
            for t in range(T_):
                if t <= T_-2:
                    for idx in range(N_ins):
                        m = gurobipy.Model(f'SDDP_{t+1}_stage_{idx+1}')
                        m.setParam('OutputFlag', OutputFlag)

                        now = m.addMVar((N_ins, 3), name='now')
                        z   = m.addMVar((N_ins, 3), name='z_prev')
                        x   = m.addVars(24, obj=-np.array(Markov_states_price[t][idx], dtype=float), name='x')
                        e_c = m.addMVar((N_ins, 24), name='met')
                        e_w = m.addMVar((N_ins, 24), name='wasted')

                        e_u_coeff = np.zeros((N_ins, 24))
                        for i in range(N_ins):
                            e_u_coeff_ = transition_matrix[t+1][idx][i]
                            e_u_coeff[i] = e_u_coeff_ * np.array(Markov_states_price[t][idx], dtype=float)

                        if (Mode in {"DRO","GMM"}) and (eta > 0) and UNMET_UNIFORM_IF_DRO:
                            base_prices = np.array(Markov_states_price[t][idx], dtype=float) 
                            e_u_uniform = np.tile((2.0*base_prices/float(N_ins))[None, :], (N_ins, 1))
                            e_u = m.addMVar((N_ins, 24), obj=e_u_uniform, name='unmet')
                        else:
                            e_u = m.addMVar((N_ins, 24), obj=2*e_u_coeff, name='unmet')

                        e_chg_0 = m.addMVar((N_ins, 24), ub=float(cap[0]), name='chg0')
                        e_chg_1 = m.addMVar((N_ins, 24), ub=float(cap[1]), name='chg1')
                        e_chg_2 = m.addMVar((N_ins, 24), ub=float(cap[2]), name='chg2')
                        e_dis_0 = m.addMVar((N_ins, 24), ub=float(cap[0]), name='dis0')
                        e_dis_1 = m.addMVar((N_ins, 24), ub=float(cap[1]), name='dis1')
                        e_dis_2 = m.addMVar((N_ins, 24), ub=float(cap[2]), name='dis2')
                        now_next_0 = m.addMVar((N_ins, 24), ub=float(cap[0]), name='lev0')
                        now_next_1 = m.addMVar((N_ins, 24), ub=float(cap[1]), name='lev1')
                        now_next_2 = m.addMVar((N_ins, 24), ub=float(cap[2]), name='lev2')

                        gamma = m.addVar(obj=1.0, lb=-gurobipy.GRB.INFINITY, name='gamma')  
                        beta  = m.addVar(obj=math.sqrt(eta), name='beta')
                        phi   = m.addVars(N_ins, obj=math.sqrt(eta)*np.ones(N_ins), name='phi')
                        mu    = m.addVars(N_ins, obj=np.sqrt(np.array(transition_matrix[t+1][idx], dtype=float)), name='mu')
                        lmbda = m.addVars(N_ins, obj=-np.sqrt(np.array(transition_matrix[t+1][idx], dtype=float)), name='lambda')
                        alpha = m.addVars(N_ins, name='alpha', lb=-1e8)
                        m.update()

                        if t == 0:
                            for n in range(N_ins):
                                for i in range(3):
                                    m.addConstr(z[n, i] == float(init_level[i]), name=f"grad_constr_{3*n+i}")
                        else:
                            for n in range(N_ins):
                                for i in range(3):
                                    m.addConstr(z[n, i] == 0.0, name=f"grad_constr_{3*n+i}")

                        for n in range(N_ins):
                            for h in range(24):
                                m.addConstr(
                                    Markov_states_power[t+1][n][h] ==
                                    e_chg_0[n, h] + e_chg_1[n, h] + e_chg_2[n, h] + e_w[n, h] + e_c[n, h],
                                    name=f"wind_const_[{n}]_[{h}]"
                                )
                                m.addConstr(
                                    x[h] == e_dis_0[n, h] + e_dis_1[n, h] + e_dis_2[n, h] + e_u[n, h] + e_c[n, h]
                                )
                            for h in range(24):
                                if h == 0:
                                    m.addConstr(now_next_0[n, 0] == rho[0]*z[n, 0] + rho_c[0]*e_chg_0[n, 0] - (1/rho_d[0])*e_dis_0[n, 0])
                                    m.addConstr(now_next_1[n, 0] == rho[1]*z[n, 1] + rho_c[1]*e_chg_1[n, 0] - (1/rho_d[1])*e_dis_1[n, 0])
                                    m.addConstr(now_next_2[n, 0] == rho[2]*z[n, 2] + rho_c[2]*e_chg_2[n, 0] - (1/rho_d[2])*e_dis_2[n, 0])
                                else:
                                    m.addConstr(now_next_0[n, h] == rho[0]*now_next_0[n, h-1] + rho_c[0]*e_chg_0[n, h] - (1/rho_d[0])*e_dis_0[n, h])
                                    m.addConstr(now_next_1[n, h] == rho[1]*now_next_1[n, h-1] + rho_c[1]*e_chg_1[n, h] - (1/rho_d[1])*e_dis_1[n, h])
                                    m.addConstr(now_next_2[n, h] == rho[2]*now_next_2[n, h-1] + rho_c[2]*e_chg_2[n, h] - (1/rho_d[2])*e_dis_2[n, h])
                            m.addConstr(now[n, 0] == now_next_0[n, 23], name="state_storage_1")
                            m.addConstr(now[n, 1] == now_next_1[n, 23], name="state_storage_2")
                            m.addConstr(now[n, 2] == now_next_2[n, 23], name="state_storage_3")

                        for n in range(N_ins):
                            wi = max(transition_matrix[t+1][idx][n], w_floor)
                            m.addConstr(
                                gamma >= alpha[n] - (1.0/math.sqrt(wi)) * (mu[n] - lmbda[n]),
                                name=f"gamma_const[{n}]"
                            )
                            m.addConstr(mu[n] + lmbda[n] == beta / math.sqrt(N_ins) + phi[n])

                        m.update()
                        M[t][idx] = m
                else:
                    for idx in range(N_ins):
                        m = gurobipy.Model(f'SDDP_{t+1}_stage_{idx+1}_last')
                        m.setParam('OutputFlag', OutputFlag)

                        now = m.addMVar((N_ins, 3), name='now')
                        z   = m.addMVar((N_ins, 3), name='z_prev')
                        x   = m.addVars(24, obj=-np.array(Markov_states_price[t][idx], dtype=float), name='x')
                        e_c = m.addMVar((N_ins, 24), name='met')
                        e_w = m.addMVar((N_ins, 24), name='wasted')

                        e_u_coeff = np.zeros((N_ins, 24))
                        for i in range(N_ins):
                            e_u_coeff_ = transition_matrix[t+1][idx][i]
                            e_u_coeff[i] = e_u_coeff_ * np.array(Markov_states_price[t][idx], dtype=float)

                        if (Mode in {"DRO","GMM"}) and (eta > 0) and UNMET_UNIFORM_IF_DRO:
                            base_prices = np.array(Markov_states_price[t][idx], dtype=float)
                            e_u_uniform = np.tile((2.0*base_prices/float(N_ins))[None, :], (N_ins, 1))
                            e_u = m.addMVar((N_ins, 24), obj=e_u_uniform, name='unmet')
                        else:
                            e_u = m.addMVar((N_ins, 24), obj=2*e_u_coeff, name='unmet')

                        e_chg_0 = m.addMVar((N_ins, 24), ub=float(cap[0]), name='chg0')
                        e_chg_1 = m.addMVar((N_ins, 24), ub=float(cap[1]), name='chg1')
                        e_chg_2 = m.addMVar((N_ins, 24), ub=float(cap[2]), name='chg2')
                        e_dis_0 = m.addMVar((N_ins, 24), ub=float(cap[0]), name='dis0')
                        e_dis_1 = m.addMVar((N_ins, 24), ub=float(cap[1]), name='dis1')
                        e_dis_2 = m.addMVar((N_ins, 24), ub=float(cap[2]), name='dis2')
                        now_next_0 = m.addMVar((N_ins, 24), ub=float(cap[0]), name='lev0')
                        now_next_1 = m.addMVar((N_ins, 24), ub=float(cap[1]), name='lev1')
                        now_next_2 = m.addMVar((N_ins, 24), ub=float(cap[2]), name='lev2')

                        for n in range(N_ins):
                            for i in range(3):
                                m.addConstr(z[n, i] == 0.0, name=f"grad_constr_{3*n+i}")

                        for n in range(N_ins):
                            for h in range(24):
                                m.addConstr(
                                    Markov_states_power[t+1][n][h] ==
                                    e_chg_0[n, h] + e_chg_1[n, h] + e_chg_2[n, h] + e_w[n, h] + e_c[n, h]
                                )
                                m.addConstr(
                                    x[h] == e_dis_0[n, h] + e_dis_1[n, h] + e_dis_2[n, h] + e_u[n, h] + e_c[n, h]
                                )
                            for h in range(24):
                                if h == 0:
                                    m.addConstr(now_next_0[n, 0] == rho[0]*z[n, 0] + rho_c[0]*e_chg_0[n, 0] - (1/rho_d[0])*e_dis_0[n, 0])
                                    m.addConstr(now_next_1[n, 0] == rho[1]*z[n, 1] + rho_c[1]*e_chg_1[n, 0] - (1/rho_d[1])*e_dis_1[n, 0])
                                    m.addConstr(now_next_2[n, 0] == rho[2]*z[n, 2] + rho_c[2]*e_chg_2[n, 0] - (1/rho_d[2])*e_dis_2[n, 0])
                                else:
                                    m.addConstr(now_next_0[n, h] == rho[0]*now_next_0[n, h-1] + rho_c[0]*e_chg_0[n, h] - (1/rho_d[0])*e_dis_0[n, h])
                                    m.addConstr(now_next_1[n, h] == rho[1]*now_next_1[n, h-1] + rho_c[1]*e_chg_1[n, h] - (1/rho_d[1])*e_dis_1[n, h])
                                    m.addConstr(now_next_2[n, h] == rho[2]*now_next_2[n, h-1] + rho_c[2]*e_chg_2[n, h] - (1/rho_d[2])*e_dis_2[n, h])
                            m.addConstr(now[n, 0] == now_next_0[n, 23])
                            m.addConstr(now[n, 1] == now_next_1[n, 23])
                            m.addConstr(now[n, 2] == now_next_2[n, 23])

                        m.update()
                        M[t][idx] = m

            print("init time:", datetime.now() - start_init)

            # ===== 도우미: Backward-cut 추가 =====
            def add_alpha_cut_to_parents(t_child, n_child, a0, bvec):
                t_parent = t_child - 1
                if t_parent < 0:
                    return
                for idx in range(N_ins):
                    m = M[t_parent][idx]
                    alpha_var = m.getVarByName(f"alpha[{n_child}]")
                    now = m.getMVarByName('now') if hasattr(m, 'getMVarByName') else None
                    if now is None:
                        now_k = [m.getVarByName(f"now[{n_child},{k}]") for k in range(3)]
                        expr = a0 + sum(bvec[k] * now_k[k] for k in range(3))
                    else:
                        expr = a0 + bvec[0]*now[n_child,0] + bvec[1]*now[n_child,1] + bvec[2]*now[n_child,2]
                    m.addConstr(alpha_var >= expr, name=f"cut_t{t_child}_n{n_child}_k")
                    m.update()

            x_bar_by_iter = []

            for it in range(NUM_ITER):
                # 1) Forward pass
                sample_path = create_sample_path(N_ins, T_, transition_matrix)
                x_bar = []
                pv = 0.0
                for t in range(T_):
                    m = M[t][sample_path[t]]
                    if t > 0:
                        prev = x_bar[t-1]
                        for i in range(3*N_ins):
                            gc = m.getConstrByName(f"grad_constr_{i}")
                            m.setAttr("RHS", gc, float(prev[i]))
                    m.optimize()
                    x = np.array([m.getVarByName(f"x[{h}]").X for h in range(24)], dtype=float)
                    price = np.array(Markov_states_price[t][sample_path[t]], dtype=float)
                    power = np.array(Markov_states_power[t+1][sample_path[t]], dtype=float)
                    if t == 0:
                        curr_storage = np.array(init_level, dtype=float)
                    revenue, curr_storage = update_storage_and_revenue(
                        x, price, power, curr_storage, cap, rho, rho_c, rho_d
                    )
                    now_vals = [m.getVarByName(f"now[{n},{i}]").X for n in range(N_ins) for i in range(3)]
                    x_bar.append(np.array(now_vals, dtype=float))
                    pv -= revenue
                x_bar_by_iter.append(x_bar)

                # 2) Backward pass
                for t in reversed(range(T_-1)):
                    forward_now = x_bar[t]
                    for n_child in range(N_ins):
                        m_child = M[t+1][n_child]
                        s0, s1, s2 = forward_now[3*n_child:3*n_child+3]
                        for k in range(3):
                            gc = m_child.getConstrByName(f"grad_constr_{3*n_child+k}")
                            if gc is not None:
                                m_child.setAttr("RHS", gc, float([s0, s1, s2][k]))
                        m_child.optimize()
                        obj = m_child.ObjVal
                        pi = []
                        for k in range(3):
                            gc = m_child.getConstrByName(f"grad_constr_{3*n_child+k}")
                            pi.append(gc.Pi if gc is not None else 0.0)
                        pi = np.array(pi, dtype=float)
                        a0 = float(obj - np.dot(pi, [s0, s1, s2]))
                        bvec = pi
                        add_alpha_cut_to_parents(t+1, n_child, a0, bvec)

            print("------------------------- OOS")
            KK = len(Markov_states_price_OOS[0])       
            Tcheck = len(Markov_states_power_OOS) - 1  

            def oos_weights_for(kk, t):
                if Mode == "GMM":
                    params = gmm_joint_params
                    if params is None:
                        return np.ones(N_ins, dtype=float) / N_ins
                    p_k  = params["p_k"]
                    mu_s = params["mu_s"]; mu_xi = params["mu_xi"]
                    S_ss = params["S_ss"]; S_scond = params["S_scond"]
                    Kmat = params["Kmat"]; S_xixi = params["S_xixi"]

                    s_vec = np.asarray(daily_wind_OOS[t, kk, :], dtype=float)
                    log_terms_den = []
                    for k in range(p_k.shape[0]):
                        Sss = S_ss[k] + COV_JITTER*np.eye(24)
                        log_terms_den.append(np.log(np.maximum(p_k[k], LOG_EPS)) +
                                             mvn.logpdf(s_vec, mean=mu_s[k], cov=Sss))
                    log_f_s = logsumexp(np.array(log_terms_den))

                    r_k_xi = per_t_cache[t]["r_k_xi"]
                    xi_mat = per_t_cache[t]["xi_mat"]

                    numer = np.zeros(N_ins, dtype=float)
                    Kc = p_k.shape[0]
                    for i in range(N_ins):
                        val_terms = np.zeros(Kc)
                        for k in range(Kc):
                            mu_cond = mu_s[k] + Kmat[k] @ (xi_mat[i] - mu_xi[k])
                            val_terms[k] = (np.log(np.maximum(r_k_xi[i, k], LOG_EPS)) +
                                            mvn.logpdf(s_vec, mean=mu_cond, cov=S_scond[k]))
                        numer[i] = np.exp(logsumexp(val_terms) - log_f_s)

                    ssum = float(numer.sum())
                    return (numer/ssum) if ssum > 0 else (np.ones(N_ins)/N_ins)

                elif Mode in {"DD", "DRO"}:
                    w = np.zeros(N_ins, dtype=float)
                    y = np.array(daily_wind_OOS[t, kk, :], dtype=float)
                    H = float(wind_H_ours[t]); S = wind_S[t]
                    for n in range(N_ins):
                        w[n] = get_kernel_likelihood_ours(daily_wind[t, n, :], y, H, S, "exp")
                    s = float(w.sum())
                    return (w / s) if s > 0 else (np.ones(N_ins) / N_ins)
                else:
                    return np.ones(N_ins, dtype=float) / N_ins

            OOS_models = []
            OOS_handles = []
            for t in range(Tcheck):
                m = M[t][0].copy()
                m.setParam('OutputFlag', 0)
                x_vars = [m.getVarByName(f"x[{h}]") for h in range(24)]
                unmet_vars = []
                for i in range(N_ins):
                    for h in range(24):
                        v = m.getVarByName(f"unmet[{i},{h}]")
                        if v is None:
                            v = m.getVarByName(f"unmet[{i*24 + h}]")
                        unmet_vars.append(v)
                mu_vars, lmbda_vars, gamma_cons = [], [], []
                for i in range(N_ins):
                    mu_i = m.getVarByName(f"mu[{i}]")
                    lam_i = m.getVarByName(f"lambda[{i}]")
                    gc_i = m.getConstrByName(f"gamma_const[{i}]")
                    if (mu_i is not None) and (lam_i is not None) and (gc_i is not None):
                        mu_vars.append(mu_i); lmbda_vars.append(lam_i); gamma_cons.append(gc_i)
                grad3 = [m.getConstrByName(f"grad_constr_{i}") for i in range(3)]
                OOS_models.append(m)
                OOS_handles.append({
                    "x": x_vars,
                    "unmet": unmet_vars,
                    "mu": mu_vars,
                    "lmbda": lmbda_vars,
                    "gamma": gamma_cons,
                    "grad3": grad3,
                })

            pvs_OOS = []
            for kk in range(KK):
                pv_OOS = 0.0
                x_bar_OOS = []
                for t in range(Tcheck):
                    m = OOS_models[t]
                    Hh = OOS_handles[t]
                    prices_24h = np.asarray(Markov_states_price_OOS[t][kk], dtype=float)
                    w_vec = oos_weights_for(kk, t).astype(float)

                    m.setAttr(gurobipy.GRB.Attr.Obj, Hh["x"], list(-prices_24h))

                    eu_coefs = []
                    if (Mode in {"DRO","GMM"}) and (eta > 0) and UNMET_UNIFORM_IF_DRO:
                        for i in range(N_ins):
                            for h in range(24):
                                eu_coefs.append(2.0 * prices_24h[h] / float(N_ins))
                    else:
                        for i in range(N_ins):
                            wi = w_vec[i]
                            for h in range(24):
                                eu_coefs.append(2.0 * wi * prices_24h[h])
                    m.setAttr(gurobipy.GRB.Attr.Obj, Hh["unmet"], eu_coefs)

                    if len(Hh["mu"]) == N_ins and len(Hh["lmbda"]) == N_ins and len(Hh["gamma"]) == N_ins:
                        for i in range(N_ins):
                            sval = math.sqrt(max(w_vec[i], w_floor))
                            Hh["mu"][i].obj    =  sval
                            Hh["lmbda"][i].obj = -sval
                            m.chgCoeff(Hh["gamma"][i], Hh["mu"][i],     0.0)
                            m.chgCoeff(Hh["gamma"][i], Hh["lmbda"][i],  0.0)
                            m.chgCoeff(Hh["gamma"][i], Hh["mu"][i],     1.0/sval)
                            m.chgCoeff(Hh["gamma"][i], Hh["lmbda"][i], -1.0/sval)

                    if t > 0:
                        for i in range(3):
                            if Hh["grad3"][i] is not None:
                                m.setAttr("RHS", Hh["grad3"][i], float(x_bar_OOS[t-1][i]))
                    m.update()
                    m.optimize()

                    x = np.array([v.X for v in Hh["x"]], dtype=float)
                    power = np.array(Markov_states_power_OOS[t+1][kk], dtype=float)
                    if t == 0:
                        curr_storage = np.array(init_level, dtype=float)
                    revenue, curr_storage = update_storage_and_revenue(
                        x, prices_24h, power, curr_storage, cap, rho, rho_c, rho_d
                    )
                    x_bar_OOS.append(curr_storage)
                    pv_OOS -= revenue
                pvs_OOS.append(pv_OOS / 1e5)

            for m in OOS_models:
                m.dispose()

            pvs_OOS = np.array(pvs_OOS, dtype=float)

            if NORMALIZE:
                pvs_OOS *= scale

            print(f"Q{q} OOS AVG: {pvs_OOS.mean():.6f}, STD: {pvs_OOS.std():.6f}")
            save_quarter_result(
                base=base, q_idx=q, Mode=Mode, N=N,
                band_mult=band_mult, eta=eta_save, pvs_OOS=pvs_OOS
            )
            del M

    print(f"[Saved under] {base}*")


Set parameter Username


Set parameter LicenseID to value 2700562


Academic license - for non-commercial use only - expires 2026-08-28


init time: 0:07:57.460305


------------------------- OOS


...

Q3 OOS AVG: -7.140185, STD: 3.348352


init time: 0:07:58.505469


------------------------- OOS


Q4 OOS AVG: -9.569712, STD: 6.900970


[Saved under] results/OH_sddp_gmm_20250908-064322*
